# Binding Affinity Prediction

This notebook allows you to predict binding affinity (pKd) for your own compounds using pre-trained models.

## Requirements
- rdkit
- pandas
- scikit-learn
- numpy
- xgboost (optional, for best model)
- lightgbm (optional)

In [ ]:
# Install dependencies if needed
# !pip install rdkit pandas scikit-learn numpy xgboost lightgbm

In [ ]:
import pandas as pd
import numpy as np
import pickle
from pathlib import Path
from rdkit import Chem
from rdkit.Chem import Descriptors
import warnings
warnings.filterwarnings('ignore')

print("Dependencies loaded successfully!")

## Step 1: Load Pre-trained Model

We provide three pre-trained models:
- **XGBoost** (best performance): R² = 0.50, RMSE = 1.33
- **Random Forest**: R² = 0.48, RMSE = 1.35
- **LightGBM**: R² = 0.47, RMSE = 1.36

In [ ]:
# Load pre-trained model
MODELS_DIR = Path('../models')

# Try to load XGBoost first (best model), fall back to Random Forest
model_data = None
model_name = None

for name in ['xgboost', 'random_forest', 'lightgbm']:
    model_path = MODELS_DIR / f'{name}_model.pkl'
    if model_path.exists():
        with open(model_path, 'rb') as f:
            model_data = pickle.load(f)
        model_name = name
        break

if model_data:
    model = model_data['model']
    feature_cols = model_data['feature_cols']
    metrics = model_data['metrics']
    
    print(f"Loaded {model_name.replace('_', ' ').title()} model")
    print(f"  Training R²: {metrics['r2']:.4f}")
    print(f"  Training RMSE: {metrics['rmse']:.4f}")
    print(f"  Training MAE: {metrics['mae']:.4f}")
    print(f"  Features: {feature_cols}")
else:
    print("ERROR: No pre-trained model found in ../models/")
    print("Please run 02_train_and_save_models.py first")

## Step 2: Define Your Compounds

Enter your compound SMILES strings below:

In [ ]:
# Enter your SMILES strings here
my_compounds = [
    "CC(C)CC1=CC=C(C=C1)C(C)C(=O)O",  # Ibuprofen
    "CC(=O)NC1=CC=C(C=C1)O",           # Paracetamol  
    "CC(=O)OC1=CC=CC=C1C(=O)O",        # Aspirin
    "CN1C=NC2=C1C(=O)N(C(=O)N2C)C",   # Caffeine
    "CC12CCC3C(C1CCC2O)CCC4=CC(=O)CCC34C",  # Testosterone
    # Add more SMILES strings as needed
]

print(f"Number of compounds to predict: {len(my_compounds)}")

## Step 3: Calculate Molecular Descriptors

In [ ]:
def calculate_descriptors(smiles):
    """Calculate molecular descriptors for a SMILES string"""
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    
    try:
        descriptors = {
            'MolWt': Descriptors.MolWt(mol),
            'LogP': Descriptors.MolLogP(mol),
            'TPSA': Descriptors.TPSA(mol),
            'NumHDonors': Descriptors.NumHDonors(mol),
            'NumHAcceptors': Descriptors.NumHAcceptors(mol),
            'NumRotatableBonds': Descriptors.NumRotatableBonds(mol),
            'NumAromaticRings': Descriptors.NumAromaticRings(mol),
            'FractionCSP3': Descriptors.FractionCSP3(mol),
            'NumHeavyAtoms': mol.GetNumHeavyAtoms(),
            'RingCount': Descriptors.RingCount(mol),
        }
        return descriptors
    except:
        return None

# Calculate descriptors for all compounds
results = []
for smiles in my_compounds:
    desc = calculate_descriptors(smiles)
    if desc:
        desc['SMILES'] = smiles
        results.append(desc)
    else:
        print(f"Warning: Could not parse SMILES: {smiles}")

df_compounds = pd.DataFrame(results)
print(f"\nCalculated descriptors for {len(df_compounds)} compounds")
df_compounds[['SMILES', 'MolWt', 'LogP', 'TPSA']]

## Step 4: Make Predictions

In [ ]:
if model_data is not None and len(df_compounds) > 0:
    # Prepare features for prediction
    X_predict = df_compounds[feature_cols].values
    
    # Make predictions
    predictions = model.predict(X_predict)
    
    # Add predictions to dataframe
    df_compounds['Predicted_pKd'] = predictions
    df_compounds['Predicted_Kd_nM'] = 10**(9 - predictions)  # Convert pKd to Kd in nM
    
    print("\n" + "="*80)
    print("PREDICTION RESULTS")
    print("="*80)
    print()
    
    for idx, row in df_compounds.iterrows():
        smiles_display = f"{row['SMILES'][:50]}..." if len(row['SMILES']) > 50 else row['SMILES']
        print(f"Compound {idx+1}: {smiles_display}")
        print(f"  Predicted pKd: {row['Predicted_pKd']:.2f}")
        print(f"  Predicted Kd:  {row['Predicted_Kd_nM']:.1f} nM")
        
        # Interpret binding strength
        if row['Predicted_pKd'] > 9:
            strength = "Very Strong (pKd > 9)"
        elif row['Predicted_pKd'] > 7:
            strength = "Strong (7 < pKd < 9)"
        elif row['Predicted_pKd'] > 5:
            strength = "Moderate (5 < pKd < 7)"
        else:
            strength = "Weak (pKd < 5)"
        print(f"  Binding: {strength}")
        print()
else:
    print("Cannot make predictions without trained model or valid compounds.")

## Step 5: Export Results

In [ ]:
# Export predictions to CSV
if 'Predicted_pKd' in df_compounds.columns:
    output_file = 'my_predictions.csv'
    df_compounds.to_csv(output_file, index=False)
    print(f"Predictions saved to: {output_file}")
    
    # Display final results table
    display_cols = ['SMILES', 'MolWt', 'LogP', 'TPSA', 'Predicted_pKd', 'Predicted_Kd_nM']
    df_compounds[display_cols]

## Important Notes

### Model Performance
- **XGBoost model**: R² = 0.50, RMSE = 1.33, MAE = 1.05 pKd units
- Trained on 16,562 compounds from PDBbind v2020
- Uses 10 molecular descriptors as features

### Model Limitations
- This is a descriptor-based model (not structure-based)
- Predictions are target-agnostic (same compound will get same prediction for any target)
- For publication-quality predictions, consider:
  - Using protein-specific models
  - Including 3D structural features
  - Using graph neural networks

### Data Quality Warning
- ~25.7% of cross-database binding data has conflicts (>10-fold difference)
- Always validate predictions experimentally
- Training data may contain measurement type mismatches (Ki vs Kd vs IC50)